In [1]:
import sys
import os
import re
import datetime

sys.path.append(os.getcwd())

from app import config
from app.modelClasses import PromptHandler, TranscriptGenerator, TeacherAgent
from app.tts import TTSProcessor
from app.pipeline import load_scenario, format_transcript, run_pipeline


/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Scenario Prep

In [2]:
scenario = (
    "A tourist enters a busy cafe and goes up to the counter to order. "
    "They don't know exactly what to order and want some advice from the server."
)
country_name = "Colombia"

### Transcript Generation

In [3]:
# Create the prompt handler and transcript generator instances
prompt_handler = PromptHandler(config.SYSTEM_PROMPT_TEMPLATE_PATH)
transcript_generator = TranscriptGenerator(config.MODEL_NAME)

# Format the prompt and generate the transcript
prompt = prompt_handler.format_prompt_scenario(scenario, country_name=country_name)
raw_transcript = transcript_generator.generate_transcript(prompt)

print("----- Raw Transcript -----")
print(raw_transcript)


----- Raw Transcript -----
Carácter 1 (Turista): ¡Hola! ¿Puedes orientarme una cosa, por favor? No estoy muy familiarizado con los platos Colombia.

Carácter 2 (Ana, la cocinera): ¡Claro que sí, amigo! ¿Qué te gustaría probar? Tenemos algunas opciones deliciosas que puedes disfrutar. ¿Te gustaría un café con arepa o quizás un sancocho o una bandeja paisa?

Carácter 1 (Turista): ¿Qué es una bandeja paisa?

Carácter 2 (Ana, la cocinera): ¡La bandeja paisa es un plato tradicional colombiano que incluye arroz, carne, platano frito y otros ingredientes deliciosos! ¿Te parece bien?

Carácter 1 (Turista): ¡Claro! Me gustaría probar una bandeja paisa. ¡Gracias!

Carácter 2 (Ana, la cocinera): ¡De nada, amigo! ¡Vente a nuestra mesa más cercana y te la traeré de inmediato! ¡Disfruta!

Carácter 1 (Turista): ¡Muchas gracias, Ana!

Carácter 2 (Ana, la cocinera): ¡De nada! ¡Vuelve pronto!

En este intercambio, el turista se acerca a un café en Colombia y se encuentra con Ana, una cocinera local. El 

In [4]:
# Create a TeacherAgent instance with the teacher prompt template
teacher_agent = TeacherAgent(
    template_path=config.TEACHER_PROMPT_TEMPLATE_PATH,
    model_name=config.MODEL_NAME,
    temperature=0.2,
)

# Generate the explained transcript based on the previously generated transcript
explained_transcript = teacher_agent.explain_transcript(raw_transcript, scenario, country_name=country_name)

print("----- Explained Transcript -----")
print(explained_transcript)


----- Explained Transcript -----
-----
Carácter 1 (Turista): ¡Hola! ¿Puedes orientarme una cosa, por favor? No estoy muy familiarizado con los platos Colombia.

Maestro: The tourist greets the server and asks for help because they're not familiar with Colombian food.

Carácter 2 (Ana, la cocinera): ¡Claro que sí, amigo! ¿Qué te gustaría probar? Tenemos algunas opciones deliciosas que puedes disfrutar. ¿Te gustaría un café con arepa o quizás un sancocho o una bandeja paisa?

Maestro: Ana, the cook, offers the tourist several delicious options, including coffee with an arepa, sancocho, or a bandeja paisa.

Carácter 1 (Turista): ¿Qué es una bandeja paisa?

Maestro: The tourist asks what a bandeja paisa is.

Carácter 2 (Ana, la cocinera): ¡La bandeja paisa es un plato tradicional colombiano que incluye arroz, carne, platano frito y otros ingredientes deliciosos! ¿Te parece bien?

Maestro: Ana explains that a bandeja paisa is a traditional Colombian dish that includes rice, meat, fried plan

In [5]:
explained_transcript = """ ----- Explained Transcript -----
-----
Juan Pablo: ¡Hola! ¿Puedes orientarme una cosa, por favor? No estoy muy familiarizado con los platos Colombia.

Maestro: The tourist greets the server and asks for help because they're not familiar with Colombian food.

Ana: ¡Claro que sí, amigo! ¿Qué te gustaría probar? Tenemos algunas opciones deliciosas que puedes disfrutar. ¿Te gustaría un café con arepa o quizás un sancocho o una bandeja paisa?

Maestro: Ana, the cook, offers the tourist several delicious options, including coffee with an arepa, sancocho, or a bandeja paisa.

Juan Pablo: ¿Qué es una bandeja paisa?

Maestro: The tourist asks what a bandeja paisa is.

Ana: ¡La bandeja paisa es un plato tradicional colombiano que incluye arroz, carne, platano frito y otros ingredientes deliciosos! ¿Te parece bien?

Maestro: Ana explains that a bandeja paisa is a traditional Colombian dish that includes rice, meat, fried plantain, and other delicious ingredients.

Juan Pablo: ¡Claro! Me gustaría probar una bandeja paisa. ¡Gracias!

Maestro: The tourist decides to try the bandeja paisa and thanks Ana.

Ana: ¡De nada, amigo! ¡Vente a nuestra mesa más cercana y te la traeré de inmediato! ¡Disfruta!

Maestro: Ana tells the tourist to go to the nearest table, and she'll bring the dish right away.
...
Lesson Summary:
- Vocabulary: bandeja paisa, café con arepa, sancocho, platano frito
- Grammar: ¿Qué te gustaría probar? (What would you like to try?)
- Cultural Takeaway: A bandeja paisa is a traditional Colombian dish that showcases the country's rich culinary heritage. """

In [6]:
unique_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
conversation_dir = os.path.join(config.GENERATED_OUTPUT_DIR, unique_id)
os.makedirs(conversation_dir, exist_ok=True)
final_audio_file = os.path.join(conversation_dir, "final_conversation.wav")

def custom_format_transcript(transcript_str):
    lines = [line.strip() for line in transcript_str.splitlines() if line.strip()]
    # Filter out header lines like "-----" or any other unwanted lines
    return [line for line in lines if not line.startswith("-----")]

transcript_lines = custom_format_transcript(explained_transcript)


In [7]:
transcript_lines

['Juan Pablo: ¡Hola! ¿Puedes orientarme una cosa, por favor? No estoy muy familiarizado con los platos Colombia.',
 "Maestro: The tourist greets the server and asks for help because they're not familiar with Colombian food.",
 'Ana: ¡Claro que sí, amigo! ¿Qué te gustaría probar? Tenemos algunas opciones deliciosas que puedes disfrutar. ¿Te gustaría un café con arepa o quizás un sancocho o una bandeja paisa?',
 'Maestro: Ana, the cook, offers the tourist several delicious options, including coffee with an arepa, sancocho, or a bandeja paisa.',
 'Juan Pablo: ¿Qué es una bandeja paisa?',
 'Maestro: The tourist asks what a bandeja paisa is.',
 'Ana: ¡La bandeja paisa es un plato tradicional colombiano que incluye arroz, carne, platano frito y otros ingredientes deliciosos! ¿Te parece bien?',
 'Maestro: Ana explains that a bandeja paisa is a traditional Colombian dish that includes rice, meat, fried plantain, and other delicious ingredients.',
 'Juan Pablo: ¡Claro! Me gustaría probar una ba

In [8]:
tts_processor = TTSProcessor(api_key=config.ELEVEN_API_KEY)

tts_processor.process_transcript(
    transcript_lines=transcript_lines,
    speaker_voices=config.SPEAKER_VOICES,
    output_dir=conversation_dir,
    final_audio_file=final_audio_file,
)

print("Audio generated for each line, and combined audio saved at:")
print(final_audio_file)

Synthesizing line 1 for 'Juan Pablo'...
Synthesizing line 2 for 'Maestro'...
Synthesizing line 3 for 'Ana'...
Synthesizing line 4 for 'Maestro'...
Synthesizing line 5 for 'Juan Pablo'...
Synthesizing line 6 for 'Maestro'...
Synthesizing line 7 for 'Ana'...
Synthesizing line 8 for 'Maestro'...
Synthesizing line 9 for 'Juan Pablo'...
Synthesizing line 10 for 'Maestro'...
Synthesizing line 11 for 'Ana'...
Synthesizing line 12 for 'Maestro'...
Line 13 is not in expected format: ...
Line 14 is not in expected format: Lesson Summary:
No speaker id found for '- Vocabulary'. Skipping line 15.
No speaker id found for '- Grammar'. Skipping line 16.
No speaker id found for '- Cultural Takeaway'. Skipping line 17.
Adding ./data/conversations/20250216_120559/line_1.wav to the combined audio...
Adding ./data/conversations/20250216_120559/line_2.wav to the combined audio...
Adding ./data/conversations/20250216_120559/line_3.wav to the combined audio...
Adding ./data/conversations/20250216_120559/line